In [ ]:
# This cell is NOT editable. Overwrite variables on your own discretion.
# Any changes other than the script code will NOT BE SAVED!
# All cells are assumed to be script code cells, unless explictly tagged as 'o9_ignore'

In [ ]:
_inp_data1 = "Select (&CWV * [Initiative].[Initiative] * [Location].[Location]*[Location].[Location Country] * [Sales Domain].[Ship To] * [Sales Domain].[Channel] * [Sales Domain].[Country] * [Sales Domain].[Region] * [Time].[Week] * [Item].[PPG] ) on row,    ({Measure.[RGM Promo Simulation Initiative Planned Base PTC],Measure.[RGM Promo Simulation Initiative Planned Promo PTC], Measure.[RGM Initiative Week Planned Promo Discount], Measure.[RGM Initiative Week Planned %ACV],Measure.[RGM Initiative Week Planned Promo %ACV], Measure.[RGM Initiative Week Planned Effective Discount],Measure.[RGM Initiative Week Planned Display Type], Measure.[RGM Initiative Week Planned Promo Mechanic]}) on column  where {~isnull (Measure.[RGM Promo Simulation Initiative Planned Base PTC])};"


# Initialize the O9DataLake with the input parameters and dataframes
# Data can be accessed with O9DataLake.get(<Input Name>)
# Overwritten values will not be reflected in the O9DataLake after initialization

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
O9DataLake.register("inp_data1",DataSource.LS, ResourceType.IBPL, _inp_data1)

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta
#
from o9helpers import user_storage_path  ##########Necessary argument to import storage path
from o9storage import cloud_storage_utils ##########Necessary argument to import cloud storage path
import os   ##########Argument to install os
import logging ##########Necessary argument to import logging package
import shutil  ##########Argument to operate with file objects
from os import path 

logger = logging.getLogger('o9_logger')
logger.info('logger initialized')

#inputs
logger.info('reading input')
RawSellout = inp_data1
logger.info(RawSellout.columns)

RawSellout['Version.[Version Name]']='sr_test'

#Output1=RawSellout 
logger.info('demo storagelayer code ended')

#"""
#bucket contains all files,bucket name can be user specified.bucket name should be changed from plug-in to plug-in.
#when you are having slice other than version(1 slice),bucket should be specified with "bucket_{}".format(df_keys['SliceKey.[Dept]']) format
#so that when saving files in storage it get's saved with slice name .
#"""
bucket = "bucket_15"   
local_storage_path = os.path.join(user_storage_path, bucket) 

######Each user will have a user storage path,joining user storage path with bucket.
test_folder_path = os.path.join(local_storage_path, "test")

#######creating a folder in loacal storage path,In this example test will be folder name in local storage path.
os.makedirs(test_folder_path)

######making test folder path as directory.
csv_path = os.path.join(test_folder_path, 'RawSellout.csv')

logger.info('#########################demo storagelayer csv_path##################')
logger.info(csv_path)
#######path to save files.
RawSellout.to_csv(csv_path,index=False) 
logger.info(RawSellout)


#"""
#Now all the files have been saved in the bucket.
##Now we have to push the bucket to cloud storage so that you can retrieve it wherever you want
#"""
try:
    logger.debug('********Before storage_push*******')
    logger.debug(os.listdir(local_storage_path))
    value = cloud_storage_utils.storage_push(bucket, local_storage_path, overwrite=True) ##########syntax to push bucket to cloud storage
    if value:
        logger.debug("storage_push successful")
    else:
        logger.debug("storage_push failed")
    logger.debug('********storage_push successful*******')
    logger.debug(os.listdir(local_storage_path))
    import glob
    logger.info('********List of files in bucket after storage push************')
    for filename in glob.iglob(local_storage_path + '**/**/*', recursive=True):
        logger.debug(filename) #################this prints all files/path  present in storage. 
        logger.info('#########################aftell push  storagelayer ##################')

except:
    logger.exception("Couldn't commit folder")

#"""
#Code for storage pull
#"""
shutil.rmtree(test_folder_path) #removing test folder path from bucket to verify storage pull
logger.debug('******** 1.List of files before storage_pull*******')
logger.debug(os.listdir(local_storage_path))
for filename in glob.iglob(local_storage_path + '**/**/*', recursive=True):
    logger.debug(filename)

logger.info('2.Reading files from storage..............................................')
local_storage_path = os.path.join(user_storage_path, bucket)
value = cloud_storage_utils.storage_pull(bucket, local_storage_path, overwrite=True)
import glob
logger.debug('********3.List of files in bucket after storage pull************')
for filename in glob.iglob(local_storage_path + '**/**/*', recursive=True):
    logger.debug(filename)

RawSellout=pd.read_csv(os.path.join(test_folder_path,'RawSellout.csv'))
logger.debug('################## 4.RawSellout after pull #####################')
logger.debug(RawSellout.columns)
logger.debug(RawSellout.head())
#output_df = None

Output1=RawSellout
logger.debug('********** 5.demo storage layer PULL Read the data from cloud storage **************')
logger.debug(os.path.join(test_folder_path,'RawSellout.csv'))


In [ ]:
# print